In [ ]:
!pip install aiogram

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.8/612.8 kB 9.1 MB/s eta 0:00:00


In [ ]:
import asyncio
import logging
import json
import gensim
import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton, InputFile
import requests

API_TOKEN = "ТУТ МОГ БЫТЬ ВАШ TOKEN"

logging.basicConfig(level=logging.INFO)
dp = Dispatcher()

with open('faq.json', encoding='utf-8') as f:
  data = json.load(f)

faq_questions = [item["question"] for item in data.get("faq", [])]
faq_answers = [item["answer"] for item in data.get("faq", [])]

# TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(faq_questions)

# Word2Vec
sentences = [q.split() for q in faq_questions]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

faq_vectors = np.array([sentence_vector(q, model) for q in faq_questions])

# Кнопки
kb = [
    [
        KeyboardButton(text="О компании"),
        KeyboardButton(text="Пожаловаться")
    ]
]

keyboard = ReplyKeyboardMarkup(
    keyboard=kb,
    resize_keyboard=True,
    input_field_placeholder="Выберите действие"
)

@dp.message(Command("start"))
async def start_command(message: types.Message):
    await message.answer("С чем вам помочь?", reply_markup=keyboard)


@dp.message(lambda message: message.text == "О компании")
async def about_company(message: types.Message):
    await message.answer("Наша компания занимается доставкой товаров по всей стране.")


@dp.message(lambda message: message.text == "Пожаловаться")
async def operator(message: types.Message):
    await message.answer("Пожалуйста, отправьте в чат скрин ошибки")

@dp.message(lambda message: message.document)
async def handle_docs(message: types.Message):
    # Асинхронная функция, которая обрабатывает полученное сообщение
    document = message.document
    # Извлекаем информацию о документе из сообщения
    file_info = f"Файл: {document.file_name}\nРазмер: {document.file_size} байт. \nВаш запрос передан специалисту " # Создаем строку с информацией о файле (имя и размер)
    await message.answer(file_info)

def get_best_answer_tfidf(query, tfidf_matrix, vectorizer, faq_answers):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, tfidf_matrix)
    best_match_idx = similarities.argmax()
    return faq_answers[best_match_idx]

def get_best_answer_word2vec(query, faq_vectors, model, faq_answers):
    query_vector = sentence_vector(query, model).reshape(1, -1)
    similarities = cosine_similarity(query_vector, faq_vectors)
    best_match_idx = similarities.argmax()
    return faq_answers[best_match_idx]

@dp.message()
async def answer_faq(message: types.Message):
    query = message.text.lower()

    # Получаем ответы от обоих методов
    best_answer_tfidf = get_best_answer_tfidf(query, tfidf_matrix, vectorizer, faq_answers)
    best_answer_word2vec = get_best_answer_word2vec(query, faq_vectors, model, faq_answers)

    # Сравниваем косинусные сходства
    query_vec_tfidf = vectorizer.transform([query])
    query_vector_word2vec = sentence_vector(query, model).reshape(1, -1)

    similarities_tfidf = cosine_similarity(query_vec_tfidf, tfidf_matrix).max()
    similarities_word2vec = cosine_similarity(query_vector_word2vec, faq_vectors).max()

    # Выбираем лучший ответ на основе максимального косинусного сходства
    if similarities_tfidf > similarities_word2vec:
        response = best_answer_tfidf
    else:
        response = best_answer_word2vec

    await message.answer(response)

async def main():
    bot = Bot(token=API_TOKEN)
    await dp.start_polling(bot)

if __name__ == "__main__":
    await main()